In [ ]:
# Importing Dependencies
import pandas as pd
import pymongo
import numpy as np
from config import api_key
from pprint import pprint
import requests
import json

## Micah's section begins

In [ ]:
oscars_file = "resources/the_oscar_award.csv"
oscars_df = pd.read_csv(oscars_file)

## Micah's section ends

## Jennifer's section begins

## Jennifer's section ends